In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from keras.models import load_model
model = load_model('classifier_model.h5')
from sklearn.externals import joblib
rfc_model = joblib.load('RFC_model.sav')

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [2]:
def quantile_25(x):
    return x.quantile(0.25)
def quantile_5(x):
    return x.quantile(0.5)
def quantile_75(x):
    return x.quantile(0.75)
def quantile_8(x):
    return x.quantile(0.8)
def quantile_9(x):
    return x.quantile(0.9)
def triptime(x):
    return x.max()

def plot_roc_curve(fpr, tpr):  
    plt.plot(fpr, tpr, color='orange', label='ROC')
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend()
    plt.show()

In [3]:
def generate_feature(df):
    df.sort_values(['bookingID','second'],inplace=True)
    radToDeg = 57.2957786
    df['roll'] = np.arctan2(df['acceleration_y'],df['acceleration_z']) * radToDeg
    df['pitch'] = np.arctan(-df['acceleration_x']/np.sqrt((df['acceleration_y']**2)+(df['acceleration_z']**2))) * radToDeg
    df['compAngleX'] = 0.93 * (df['roll'] + df['gyro_x']/131 ) + 0.07 * df['roll']
    df['compAngleY'] = 0.93 * (df['pitch'] + df['gyro_x']/131 ) + 0.07 * df['pitch']
    df['Bearing_diff']=df['Bearing'].diff()
    df.loc[df['second']==0,['Bearing_diff']]= 0
    df['Speed_diff']=df['Speed'].diff()
    df.loc[df['second']==0,['Speed_diff']]= 0
    df['distance'] = df['Speed'] * df['second'].diff()
    df['distance'][df['second'] == 0] = 0
    model_df_1 = df.pivot_table(index='bookingID',values = ['distance'], aggfunc=sum)
    model_df_2 = df.pivot_table(index='bookingID',values = ['second'], aggfunc=max)
    model_df_3 = df.pivot_table(index='bookingID',values=['Speed','Bearing','compAngleX','compAngleY','Bearing_diff','Speed_diff'],aggfunc=[quantile_25,quantile_5,quantile_75,quantile_8,quantile_9])
    model_df_4 = df.pivot_table(index='bookingID',values=['Speed','Bearing','compAngleX','compAngleY','Bearing_diff','Speed_diff'],aggfunc=[np.mean,np.std,np.min,np.max])
    model_df = model_df_1.join([model_df_2,model_df_3,model_df_4])
    return model_df


In [70]:
def predict(x):
    x_feature = generate_feature(x)
    keras_prediction = model.predict(x_feature)
    keras_prediction = [round(i[0]) for i in keras_prediction]
    rfc_prediction = rfc_model.predict(x_feature)
    join_df = pd.DataFrame({'bookingID':test_feature.index,'Keras_prediction':keras_prediction,'RFC_prediction':rfc_prediction})
    return join_df

**Load the test data**

In [71]:
test = pd.read_csv('test_trip2.csv')

**Make prediction**

In [72]:
predict(test)

C:\Users\minhcq\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


,bookingID,Keras_prediction,RFC_prediction
0,1.000000e+00,0.0,0
1,1.073742e+12,0.0,1
